This is a data analysis of Reddit posts pertaining to the effects of AI on the job market. We start by importing the libraries that we will need.

In [3]:
import pandas as pd
import matplotlib as plt
import re

from collections import Counter

Next, we import our dataset into Pandas for processing.

In [5]:
df = pd.read_csv('ai_automation_job_market_data200_edit.csv')

pd.set_option('display.max_columns', None) # Avoid truncation

Now it's time for initial data exploration

In [7]:
df

,Title,Selftext,Score,Number of Comments,Created UTC,Unnamed: 5
0,CMV: People who think that most jobs are going...,I have been lucky enough to have a career path...,705,511.0,1.708275e+09,NaN
1,Is this the last presidential election in US h...,[Uber is partnering with Chinese firm BYD to b...,406,230.0,1.727364e+09,NaN
2,What will the stock market do if ai creates gr...,I know nobody has a crystal ball and no one ca...,36,166.0,1.726364e+09,NaN
3,"I'm Nick Kolakowski, Senior Editor at Dice. AM...","Hi! I?檓 Nick Kolakowski, the Senior Editor of ...",102,143.0,1.718886e+09,NaN
4,The Elephant In the Room with AI art and AI Au...,"\n\nAs I see it, the main issue with AI art a...",165,334.0,1.672166e+09,NaN
...,...,...,...,...,...,...
173,What does the future of our work look like in ...,I'm sure we've all been thinking a lot lately ...,18,79.0,1.728255e+09,NaN
174,"Blackmen, is this dystopian reality possible?",Is this dystopian reality possible?\n\n\t??Mas...,10,42.0,1.739900e+09,NaN
175,Learnings from my Experience in USA: [BTech ->...,**TLDR:**\n\n1. US immigration and job landsca...,128,40.0,1.733100e+09,NaN
176,The 15 Best (Free to Use) AI Tools for Creatin...,While we wait for ChatGPT to roll out its own ...,458,64.0,1.695663e+09,NaN


The first thing we notice is there's an unnecessary column with NaN (non-numerical, invalid) values. In addition, the timestamp column in UTC isn't in an easily human-readable format, so it needs to be displayed as a human-readable date even though UTC is easier to use for data processing.

In [9]:
df = df.drop('Unnamed: 5', axis=1)

In [10]:
df

,Title,Selftext,Score,Number of Comments,Created UTC
0,CMV: People who think that most jobs are going...,I have been lucky enough to have a career path...,705,511.0,1.708275e+09
1,Is this the last presidential election in US h...,[Uber is partnering with Chinese firm BYD to b...,406,230.0,1.727364e+09
2,What will the stock market do if ai creates gr...,I know nobody has a crystal ball and no one ca...,36,166.0,1.726364e+09
3,"I'm Nick Kolakowski, Senior Editor at Dice. AM...","Hi! I?檓 Nick Kolakowski, the Senior Editor of ...",102,143.0,1.718886e+09
4,The Elephant In the Room with AI art and AI Au...,"\n\nAs I see it, the main issue with AI art a...",165,334.0,1.672166e+09
...,...,...,...,...,...
173,What does the future of our work look like in ...,I'm sure we've all been thinking a lot lately ...,18,79.0,1.728255e+09
174,"Blackmen, is this dystopian reality possible?",Is this dystopian reality possible?\n\n\t??Mas...,10,42.0,1.739900e+09
175,Learnings from my Experience in USA: [BTech ->...,**TLDR:**\n\n1. US immigration and job landsca...,128,40.0,1.733100e+09
176,The 15 Best (Free to Use) AI Tools for Creatin...,While we wait for ChatGPT to roll out its own ...,458,64.0,1.695663e+09


Having cleaned our dataset, the next thing to do is check the frequency of specific key terms in our post data, both the titles and main text. We will load our terms from an external file to make this easier to maintain.

Example reference: [GeeksForGeeks Python Read Text File into List or Array - Using List Comprehension](https://www.geeksforgeeks.org/python-read-text-file-into-list-or-array/#using-list-comprehension) for guidance on loading lines from an external text file.

In [12]:
key_terms=[]

with open('terms.txt', 'r') as file:
    key_terms = [line.strip() for line in file]

print(key_terms)

['AI', 'artificial intelligence', 'LLM', 'automation', 'impact', 'unemployment', 'risk', 'loss', 'layoff', 'fired', 'redundant', 'struggling', 'obsolete', 'retrain', 'stress', 'concerned', 'scared', 'frustrated', 'hopeless', 'overwhelmed']


This function is completely hand-written, because an attempt to use [Anaconda Assistant](https://www.anaconda.com/capability/anaconda-assistant) to generate an example resulted in a function with too many parameters, redundant comments, and variable names that don't flow particularly well with the conventions used elsewhere in this notebook. It also turned out to be highly inefficient and thus a new approach based on the advantages of the Pandas library has been implemented. As a result, the cell with AI-generated code was removed and replaced with this hand-written function. Only the most superficial code to lower the case of all characters in a string was reused from generated code. [Anaconda Assistant](https://www.anaconda.com/capability/anaconda-assistant) has been turned off for this notebook and will not be used for anything else.

This function is to be reused for both titles and post bodies.

In [14]:
def string_frequency(column='Title'):
    strings_lower = df[column].str.lower()

    # word_counts = pd.Series(split_string).value_counts()

    dfs = []

    for string in strings_lower:
        word_counts = Counter(strings_lower)
        new_df = pd.DataFrame.from_dict(word_counts, orient='index')
        dfs.append(new_df)

    return dfs

Verify the columns that will have their text processed.

In [16]:
df["Title"]

0      CMV: People who think that most jobs are going...
1      Is this the last presidential election in US h...
2      What will the stock market do if ai creates gr...
3      I'm Nick Kolakowski, Senior Editor at Dice. AM...
4      The Elephant In the Room with AI art and AI Au...
                             ...                        
173    What does the future of our work look like in ...
174        Blackmen, is this dystopian reality possible?
175    Learnings from my Experience in USA: [BTech ->...
176    The 15 Best (Free to Use) AI Tools for Creatin...
177                       Been Unemployed for 1 Year! ?槓
Name: Title, Length: 178, dtype: object

In [17]:
df["Selftext"]

0      I have been lucky enough to have a career path...
1      [Uber is partnering with Chinese firm BYD to b...
2      I know nobody has a crystal ball and no one ca...
3      Hi! I?檓 Nick Kolakowski, the Senior Editor of ...
4       \n\nAs I see it, the main issue with AI art a...
                             ...                        
173    I'm sure we've all been thinking a lot lately ...
174    Is this dystopian reality possible?\n\n\t??Mas...
175    **TLDR:**\n\n1. US immigration and job landsca...
176    While we wait for ChatGPT to roll out its own ...
177    It's official... I've been unemployed for 1 ye...
Name: Selftext, Length: 178, dtype: object

In [18]:
title_df=string_frequency()

In [19]:
title_df

,0
cmv: people who think that most jobs are going to be replaced by ai in the next couple decades have very little understanding of how modern societies function and have a skewed view of macroeconomics.,1
is this the last presidential election in us history that won't be dominated by the issue of reorganizing society as free-market human jobs disappear to automation?,1
what will the stock market do if ai creates growth in businesses but also makes lots of people lose their jobs due to automation?,1
"i'm nick kolakowski, senior editor at dice. ama about the tech job market, ai, and more!",1
the elephant in the room with ai art and ai automation generally,1
...,...
what does the future of our work look like in the age of ai tools,1
"blackmen, is this dystopian reality possible?",1
"learnings from my experience in usa: [btech -> swe [msft india] -> ms -> mle 2 [tiktok, meta]",1
"the 15 best (free to use) ai tools for creating websites, presentations, graphics, uis, photos, and more",1


In [20]:
selftext_df=string_frequency('Selftext')

In [21]:
display(selftext_df)

0
i have been lucky enough to have a career path ...  1
[uber is partnering with chinese firm byd to bu...  1
i know nobody has a crystal ball and no one can...  3
hi! i?檓 nick kolakowski, the senior editor of c...  1
 \n\nas i see it, the main issue with ai art an...  1
...                                                ..
i'm sure we've all been thinking a lot lately a...  1
is this dystopian reality possible?\n\n\t??mass...  1
**tldr:**\n\n1. us immigration and job landscap...  1
while we wait for chatgpt to roll out its own o...  1
it's official... i've been unemployed for 1 yea...  1

[175 rows x 1 columns]

Our raw dataframes indicate a disproportionate emphasis on automation, however we must use graphs to make it easier to visualize frequency.